<a href="https://colab.research.google.com/github/vitaldavi/RAG-LLMs/blob/main/An%C3%A1lise_Dovish_Hawkish_das_Atas_do_COPOM_usando_LangChain_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar bibliotecas
!pip install langchain_community langchain-google-genai langchain pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 998.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.3.2
    Un

In [ ]:
# Bibliotecas ----
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.schema.prompt_template import format_document
from langchain.schema import StrOutputParser
import os

In [ ]:
# Autenticação na API do Google
os.environ["GOOGLE_API_KEY"] = "xxxxxxx" # COLOQUE AQUI SUA CHAVE

In [ ]:
# Coleta de dados ----
pdf = PyPDFLoader("https://www.bcb.gov.br/content/copom/atascopom/Copom260-not20240131260.pdf")
ata = pdf.load_and_split()
ata

[Document(page_content='1    bcb.gov.br  \n \n  \n \n \n \n  \n \n  \n \n \n \n \n  \n \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nAta da Reunião do  \nComitê de Política Monetária  — Copom  \n30 e 31 de janei ro de 202 4 260ª', metadata={'source': 'https://www.bcb.gov.br/content/copom/atascopom/Copom260-not20240131260.pdf', 'page': 0}),
 Document(page_content='260ª Reunião  \nCOPOM  \n30 e 31 de janeiro  \n  \n2    bcb.gov.br   \nOs membros do Copom analisaram a evolução recente e as perspectivas para a economia brasileira \ne para a economia internacional, no context o do regime de política monetária, cujo objetivo é \natingir as metas fixadas pelo Conselho Monetário Nacional para a inflação.    \nData:  \n30 e 31  de janeiro  de 2024 \n  \nLocal:  Sala s de reuniões do 8º andar (30/1  e 31/1 – manhã) e do 20º andar ( 31/1  – tarde ) do \nEdifício -sede do Banco Central do Brasil – Brasília – DF \n  \nHorários de início  \ne término : 30 de janei ro: 10h06 – 12h26; 14h 1

In [ ]:
# Definição do modelo LLM e Engenharia de prompt ----
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

doc_prompt = PromptTemplate.from_template("{page_content}") # extrai texto do PDF

llm_prompt = PromptTemplate.from_template("""
Você é um economista chefe experiente em análise de política monetária, tendo
trabalhado em fundos de investimentos e grandes bancos, além de ser PhD em
economia.

Sua tarefa é ler a ata a seguir da Reunião do Comitê de Política Monetária (Copom)
do Banco Central do Brasil e classificar o documento em hawkish ou dovish:

"{ata}"

Classificação:""") # prompt para o Gemini

# Combinar prompt, texto da ata e modelo
classificador = (
    {"ata": lambda ata: "\n\n".join(
      format_document(pagina, doc_prompt) for pagina in ata
      )}
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Classificação do modelo LLM ----
classificador.invoke(ata[2:8])

'Dovish'